In [5]:
!pip install cpmaxToolbox

You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import cpmaxToolbox.FilterTool as ft
import matplotlib.pyplot as plt
import time
%matplotlib qt

import numpy as np

In [2]:
datnam = "Bremerhaven.csv"
fs = 512
meas_id = 2

In [3]:
print("read file")
t0 = time.time()
df = pd.read_csv(datnam)
df = df[df["Measurement_ID"] == meas_id]
df_dict = {
    "original":df
}
print(f"{time.time()-t0}s")

read file
0.7042317390441895s


In [4]:
ax_list = ["AxialSignal", "RadialSignal", "TorsionalSignal"]
trig = "RotationTrigger"
thres = 1000
mean_diff = 20
timing = {}

t0 = time.time()
df_capped = ft.cap_thres(df, ax_list, thres, False)
df_dict["capped_thres"] = df_capped
timing["capped_thres"] = time.time()-t0

t0 = time.time()
df_filt_thres = ft.filt_rot_thres(df, ax_list, trig, thres, False, True)
df_dict["filtered_thres"] = df_filt_thres
tt = timing["filtered_thres"] = time.time()-t0

t0 = time.time()
df_filt_mean = ft.filt_rot_mean(df, ax_list, trig, mean_diff, False, True)
df_dict["filtered_mean"] = df_filt_mean
timing["filtered_mean"] = time.time()-t0

t0 = time.time()
df_filt_both = ft.filt_rot_mean(df_filt_thres, ax_list, trig, mean_diff, False, True)
df_dict["filtered_both"] = df_filt_both
timing["filtered_both"] = time.time()-t0 + tt

_=[print(f"{k} - {t:.2f}s") for k,t in timing.items()]


Output()

Output()

Output()

capped_thres - 0.08s
filtered_thres - 0.25s
filtered_mean - 0.20s
filtered_both - 0.42s


In [5]:
fig, axs = plt.subplots(2, len(df_dict), sharex=True, sharey="row")
plt.subplots_adjust(top=0.962, bottom=0.039, left=0.042, right=0.992, hspace=0.098, wspace=0.051)

for i, (k, dfi) in enumerate(df_dict.items()):
    print(i, k)
    axs[0,i].title.set_text(f'Radial - {k} - {100*len(dfi)/len(df):.2f}%')
    axs[0,i].plot(dfi.index/fs, thres*dfi.RotationTrigger, linewidth=0.5)
    axs[0,i].plot(dfi.index/fs, dfi.RadialSignal, linewidth=0.5)
    axs[0,i].grid(True)

    axs[1,i].title.set_text(f'Axial - {k} - {100*len(dfi)/len(df):.2f}%')
    axs[1,i].plot(dfi.index/fs, thres*dfi.RotationTrigger, linewidth=0.5)
    axs[1,i].plot(dfi.index/fs, df_dict[k].AxialSignal, linewidth=0.5)
    axs[1,i].grid(True)
axs[0,0].axis(xmin=df.index.min()/fs, xmax=df.index.max()/fs)

figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()

0 original
1 capped_thres
2 filtered_thres
3 filtered_mean
4 filtered_both


In [ ]:
axis_dict = {
    "AxialSignal": "Axial", 
    "RadialSignal": "Radial", 
    "TorsionalSignal": "Torsional", 
    "RotationTrigger": "Trigger"
    }
fnam = datnam.replace(".csv", f"_{meas_id}.txt")
ft.to_vibA_import(df_dict["filtered_both"], fnam, fs, axis_dict, 1)